In [1]:
from transformers import DistilBertForQuestionAnswering, AutoTokenizer, pipeline, Trainer
from datasets import load_dataset

In [2]:
squad = load_dataset("squad", split="train[:5000]")
wiki_qa = load_dataset("wiki_qa", split="train[:5000]")

Extracting data files: 100%|█████████████████████| 2/2 [00:00<00:00, 674.11it/s]


Dataset squad downloaded and prepared to /Users/natepruitt/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


Dataset wiki_qa downloaded and prepared to /Users/natepruitt/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c. Subsequent calls will reuse this data.


In [15]:
squad = squad.train_test_split(test_size=0.2)
wiki_qa = wiki_qa.train_test_split(test_size=0.2)

In [18]:
squad["train"]["answers"]

[{'text': ['10'], 'answer_start': [115]},
 {'text': ['Carnaval'], 'answer_start': [138]},
 {'text': ['miscarriage'], 'answer_start': [617]},
 {'text': ["B'Day"], 'answer_start': [91]},
 {'text': ['Fiorello La Guardia'], 'answer_start': [292]},
 {'text': ['Nintendo'], 'answer_start': [177]},
 {'text': ['activewear'], 'answer_start': [299]},
 {'text': ['56.4 million'], 'answer_start': [220]},
 {'text': ['8,491,079'], 'answer_start': [95]},
 {'text': ['1525'], 'answer_start': [133]},
 {'text': ['A Mongol dynasty of China'], 'answer_start': [189]},
 {'text': ['Hyper'], 'answer_start': [1524]},
 {'text': ['various natural compounds'], 'answer_start': [100]},
 {'text': ['Heat'], 'answer_start': [247]},
 {'text': ['during filming.'], 'answer_start': [138]},
 {'text': ['201,000'], 'answer_start': [511]},
 {'text': ['the Harlem Renaissance'], 'answer_start': [70]},
 {'text': ['four'], 'answer_start': [161]},
 {'text': ['PAL'], 'answer_start': [7]},
 {'text': ['wrote music'], 'answer_start': [73

In [55]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
questions = [q.strip() for q in squad["train"]["question"]]
inputs = tokenizer(
        questions,
        squad["train"]["context"],
        max_length=384,
        return_offsets_mapping=True,
        padding="max_length",
    )


In [50]:
len(inputs["offset_mapping"])
len(squad["train"]["context"])

4000

In [28]:
squad["train"]["context"]

["During the December 2014 press conference announcing the start of filming, Aston Martin and Eon unveiled the new DB10 as the official car for the film. The DB10 was designed in collaboration between Aston Martin and the filmmakers, with only 10 being produced especially for Spectre as a celebration of the 50th anniversary of the company's association with the franchise. Only eight of those 10 were used for the film, however; the remaining two were used for promotional work. After modifying the Jaguar C-X75 for the film, Williams F1 carried the 007 logo on their cars at the 2015 Mexican Grand Prix, with the team playing host to the cast and crew ahead of the Mexican premiere of the film.",
 "Chopin's qualities as a pianist and composer were recognized by many of his fellow musicians. Schumann named a piece for him in his suite Carnaval, and Chopin later dedicated his Ballade No. 2 in F major to Schumann. Elements of Chopin's music can be traced in many of Liszt's later works. Liszt la

In [29]:
squad["train"]["answers"]

[{'text': ['10'], 'answer_start': [115]},
 {'text': ['Carnaval'], 'answer_start': [138]},
 {'text': ['miscarriage'], 'answer_start': [617]},
 {'text': ["B'Day"], 'answer_start': [91]},
 {'text': ['Fiorello La Guardia'], 'answer_start': [292]},
 {'text': ['Nintendo'], 'answer_start': [177]},
 {'text': ['activewear'], 'answer_start': [299]},
 {'text': ['56.4 million'], 'answer_start': [220]},
 {'text': ['8,491,079'], 'answer_start': [95]},
 {'text': ['1525'], 'answer_start': [133]},
 {'text': ['A Mongol dynasty of China'], 'answer_start': [189]},
 {'text': ['Hyper'], 'answer_start': [1524]},
 {'text': ['various natural compounds'], 'answer_start': [100]},
 {'text': ['Heat'], 'answer_start': [247]},
 {'text': ['during filming.'], 'answer_start': [138]},
 {'text': ['201,000'], 'answer_start': [511]},
 {'text': ['the Harlem Renaissance'], 'answer_start': [70]},
 {'text': ['four'], 'answer_start': [161]},
 {'text': ['PAL'], 'answer_start': [7]},
 {'text': ['wrote music'], 'answer_start': [73

In [57]:
tokenizer.tokenize(squad["train"]["context"][0])

['during',
 'the',
 'december',
 '2014',
 'press',
 'conference',
 'announcing',
 'the',
 'start',
 'of',
 'filming',
 ',',
 'aston',
 'martin',
 'and',
 'e',
 '##on',
 'unveiled',
 'the',
 'new',
 'db',
 '##10',
 'as',
 'the',
 'official',
 'car',
 'for',
 'the',
 'film',
 '.',
 'the',
 'db',
 '##10',
 'was',
 'designed',
 'in',
 'collaboration',
 'between',
 'aston',
 'martin',
 'and',
 'the',
 'filmmakers',
 ',',
 'with',
 'only',
 '10',
 'being',
 'produced',
 'especially',
 'for',
 'spec',
 '##tre',
 'as',
 'a',
 'celebration',
 'of',
 'the',
 '50th',
 'anniversary',
 'of',
 'the',
 'company',
 "'",
 's',
 'association',
 'with',
 'the',
 'franchise',
 '.',
 'only',
 'eight',
 'of',
 'those',
 '10',
 'were',
 'used',
 'for',
 'the',
 'film',
 ',',
 'however',
 ';',
 'the',
 'remaining',
 'two',
 'were',
 'used',
 'for',
 'promotional',
 'work',
 '.',
 'after',
 'modifying',
 'the',
 'jaguar',
 'c',
 '-',
 'x',
 '##75',
 'for',
 'the',
 'film',
 ',',
 'williams',
 'f1',
 'carried',

In [56]:
inputs["offset_mapping"][0]

[(0, 0),
 (0, 3),
 (4, 8),
 (9, 11),
 (11, 13),
 (13, 14),
 (14, 15),
 (16, 20),
 (21, 29),
 (29, 30),
 (0, 0),
 (0, 6),
 (7, 10),
 (11, 19),
 (20, 24),
 (25, 30),
 (31, 41),
 (42, 52),
 (53, 56),
 (57, 62),
 (63, 65),
 (66, 73),
 (73, 74),
 (75, 80),
 (81, 87),
 (88, 91),
 (92, 93),
 (93, 95),
 (96, 104),
 (105, 108),
 (109, 112),
 (113, 115),
 (115, 117),
 (118, 120),
 (121, 124),
 (125, 133),
 (134, 137),
 (138, 141),
 (142, 145),
 (146, 150),
 (150, 151),
 (152, 155),
 (156, 158),
 (158, 160),
 (161, 164),
 (165, 173),
 (174, 176),
 (177, 190),
 (191, 198),
 (199, 204),
 (205, 211),
 (212, 215),
 (216, 219),
 (220, 230),
 (230, 231),
 (232, 236),
 (237, 241),
 (242, 244),
 (245, 250),
 (251, 259),
 (260, 270),
 (271, 274),
 (275, 279),
 (279, 282),
 (283, 285),
 (286, 287),
 (288, 299),
 (300, 302),
 (303, 306),
 (307, 311),
 (312, 323),
 (324, 326),
 (327, 330),
 (331, 338),
 (338, 339),
 (339, 340),
 (341, 352),
 (353, 357),
 (358, 361),
 (362, 371),
 (371, 372),
 (373, 377),
 (3

In [48]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])

In [54]:
questions[0]

"How many DB10's were produced?"